<center><u><b><h1 style="color:Red;">Titanic : Machine Learning from Disaster</u></b></h1></center>

<b>Loading Data</b>

In [63]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load data
train_df = pd.read_csv(r"C:\Kaggles\Titanic\Dataset\train.csv")
test_df = pd.read_csv(r"C:\Kaggles\Titanic\Dataset\test.csv")
test_ids = test_df["PassengerId"]

# Combine train and test datasets to handle them together
combine = [train_df, test_df]

# Combine datasets for preprocessing
test_df['Survived'] = np.nan  # Add a dummy 'Survived' column to align columns
combined_df = pd.concat([train_df, test_df])

print(train_df.shape, test_df.shape, combined_df.shape)
combined_df.head()

(891, 12) (418, 12) (1309, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<b>Removing Unrequired Columns</b>

In [64]:
combined_df.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1, inplace=True)
combined_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0.0,3,male,22.0,1,0,7.2500,S
1,1.0,1,female,38.0,1,0,71.2833,C
2,1.0,3,female,26.0,0,0,7.9250,S
3,1.0,1,female,35.0,1,0,53.1000,S
4,0.0,3,male,35.0,0,0,8.0500,S


<b>Handling NaN values</b>

In [65]:
combined_df['Age'].fillna(combined_df['Age'].median(), inplace=True)
combined_df['Embarked'].fillna(combined_df['Embarked'].mode()[0], inplace=True)
combined_df['Fare'].fillna(combined_df['Fare'].median(), inplace=True)

<b>Feature Engineering</b>

In [66]:
combined_df['FamilySize'] = combined_df['SibSp'] + combined_df['Parch'] + 1
combined_df['IsAlone'] = 1
combined_df.loc[combined_df['FamilySize'] > 1, 'IsAlone'] = 0

In [67]:
combined_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,0.0,3,male,22.0,1,0,7.2500,S,2,0
1,1.0,1,female,38.0,1,0,71.2833,C,2,0
2,1.0,3,female,26.0,0,0,7.9250,S,1,1
3,1.0,1,female,35.0,1,0,53.1000,S,2,0
4,0.0,3,male,35.0,0,0,8.0500,S,1,1


In [68]:
combined_df.drop(["SibSp", "Parch"], axis=1, inplace=True)

In [69]:
combined_df.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,IsAlone
0,0.0,3,male,22.0,7.2500,S,2,0
1,1.0,1,female,38.0,71.2833,C,2,0
2,1.0,3,female,26.0,7.9250,S,1,1
3,1.0,1,female,35.0,53.1000,S,2,0
4,0.0,3,male,35.0,8.0500,S,1,1


<b>Handling Categorical Features</b>

In [70]:
# Convert categorical features to numeric
combined_df['Sex'] = combined_df['Sex'].map({'female': 1, 'male': 0}).astype(int)
combined_df['Embarked'] = combined_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)

<b>Seperating the Data back</b>

In [71]:
# Separate back into train and test sets
train_df = combined_df[combined_df['Survived'].notna()]
test_df = combined_df[combined_df['Survived'].isna()].drop('Survived', axis=1)

In [72]:
X_train_full = train_df.drop('Survived', axis=1)
y_train_full = train_df['Survived']
X_test = test_df.copy()

# Split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.12, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Build the model
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy}')

# Detailed classification report
print("Classification Report:")
print(classification_report(y_val, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_val, y_pred))


Validation Accuracy: 0.8411214953271028
Classification Report:
              precision    recall  f1-score   support

         0.0       0.84      0.91      0.87        65
         1.0       0.84      0.74      0.78        42

    accuracy                           0.84       107
   macro avg       0.84      0.82      0.83       107
weighted avg       0.84      0.84      0.84       107

Confusion Matrix:
[[59  6]
 [11 31]]


<b>Ouput for test Dataset</b>

In [73]:
# Make predictions on the test set
predictions = model.predict(X_test).astype(int)

# Create submission file
submission = pd.DataFrame({
    "PassengerId": pd.read_csv(r"C:\Kaggles\Titanic\Dataset\test.csv")["PassengerId"],
    "Survived": predictions
})

submission.to_csv(r"C:\Kaggles\Titanic\Submissions\Submission_2.csv", index=False)